In [1]:
import os
import glob
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import upsetplot
#import humanize

pd.set_option('display.max_rows', 500)

import pybedtools as pbt 
pbt.set_bedtools_path('/mnt/BioHome/jreyna/software/anaconda3/envs/hic_tls/bin/')
#pbt.set_tempdir('/mnt/bioadhoc-temp/Groups/vd-ay/jreyna-temp/scratch/')


os.chdir('/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/t1d-loop-catalog/')
outdir = 'results/hg38/finemapping/sgls/summary/'
os.makedirs(outdir, exist_ok=True)

In [2]:
# 35 different sample names
fns = glob.glob('results/hg38/finemapping/sgls/*.5000/*.5000.finemap_sgls.tsv')

In [3]:
# each sample has biological duplicates
for fn in fns:
    if "CD8_Naive_1815-RH-1" in fn:
        print(fn)

results/hg38/finemapping/sgls/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2.5000/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b2.5000.finemap_sgls.tsv
results/hg38/finemapping/sgls/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1.5000/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b1.5000.finemap_sgls.tsv
results/hg38/finemapping/sgls/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b3.5000/CD8_Naive_1815-RH-1.phs001703v3p1.Homo_Sapiens.H3K27ac.b3.5000.finemap_sgls.tsv


#### Exmaple of treating one fn

In [4]:
sample = fns[0]
sample_name = (sample.split("/")[-1]).split(".")[0]
df = pd.read_table(sample)
df["sample_name"] = sample_name
df["summary_id"] = df["chr_snp"].astype(str) + "_" + df["start_snp"].astype(str) + "_" \
                    + df["geneid"].astype(str) + "_" + df["startA_loop"].astype(str) + "_" \
                    + df["startB_loop"].astype(str)
df

,chrA_loop,startA_loop,endA_loop,chrB_loop,startB_loop,endB_loop,-log10_qval_loop,chr_snp,start_snp,end_snp,...,ppa_snp,snp_anchor,chr_gene,start_gene,end_gene,genename,geneid,strand,sample_name,summary_id
0,1,112615000,112620000,1,113905000,113910000,6.195724,1,113909957,113909958,...,0.058538,AnchorB,1,112619804,112619805,CAPZA1,ENSG00000116489,+,CD8_Naive_1815-RH-1,1_113909957_ENSG00000116489_112615000_113905000
1,1,113905000,113910000,1,113925000,113930000,18.295846,1,113909957,113909958,...,0.058538,AnchorA,1,113929323,113929324,HIPK1,ENSG00000163349,+,CD8_Naive_1815-RH-1,1_113909957_ENSG00000163349_113905000_113925000
2,1,113905000,113910000,1,113925000,113930000,18.295846,1,113909957,113909958,...,0.058538,AnchorA,1,113929491,113929492,HIPK1-AS1,ENSG00000235527,-,CD8_Naive_1815-RH-1,1_113909957_ENSG00000235527_113905000_113925000
3,11,128295000,128300000,11,128520000,128525000,11.801121,11,128298735,128298736,...,0.130943,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128298735_ENSG00000276176_128295000_128520000
4,11,128300000,128305000,11,128520000,128525000,53.208704,11,128302574,128302575,...,0.122752,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128302574_ENSG00000276176_128300000_128520000
5,11,128300000,128305000,11,128520000,128525000,53.208704,11,128302747,128302748,...,0.125901,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128302747_ENSG00000276176_128300000_128520000
6,11,128300000,128305000,11,128520000,128525000,53.208704,11,128304885,128304886,...,0.127477,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128304885_ENSG00000276176_128300000_128520000
7,11,128300000,128305000,11,128525000,128530000,12.543043,11,128302574,128302575,...,0.122752,AnchorA,11,128526141,128526142,ETS1-AS1,ENSG00000254588,+,CD8_Naive_1815-RH-1,11_128302574_ENSG00000254588_128300000_128525000
8,11,128300000,128305000,11,128525000,128530000,12.543043,11,128302747,128302748,...,0.125901,AnchorA,11,128526141,128526142,ETS1-AS1,ENSG00000254588,+,CD8_Naive_1815-RH-1,11_128302747_ENSG00000254588_128300000_128525000
9,11,128300000,128305000,11,128525000,128530000,12.543043,11,128304885,128304886,...,0.127477,AnchorA,11,128526141,128526142,ETS1-AS1,ENSG00000254588,+,CD8_Naive_1815-RH-1,11_128304885_ENSG00000254588_128300000_128525000


#### Treating all fns

In [5]:
dfs = []
for fn in fns:
    sample_name = (fn.split("/")[-1]).split(".")[0]
    df = pd.read_table(sample)
    df["sample_name"] = sample_name
    df["summary_id"] = df["chr_snp"].astype(str) + "_" + df["start_snp"].astype(str) + "_" \
                        + df["geneid"].astype(str) + "_" + df["startA_loop"].astype(str) + "_" \
                        + df["startB_loop"].astype(str)
    dfs.append(df)
    
all_df = pd.concat(dfs, ignore_index=True)

In [6]:
all_df

,chrA_loop,startA_loop,endA_loop,chrB_loop,startB_loop,endB_loop,-log10_qval_loop,chr_snp,start_snp,end_snp,...,ppa_snp,snp_anchor,chr_gene,start_gene,end_gene,genename,geneid,strand,sample_name,summary_id
0,1,112615000,112620000,1,113905000,113910000,6.195724,1,113909957,113909958,...,0.058538,AnchorB,1,112619804,112619805,CAPZA1,ENSG00000116489,+,CD8_Naive_1815-RH-1,1_113909957_ENSG00000116489_112615000_113905000
1,1,113905000,113910000,1,113925000,113930000,18.295846,1,113909957,113909958,...,0.058538,AnchorA,1,113929323,113929324,HIPK1,ENSG00000163349,+,CD8_Naive_1815-RH-1,1_113909957_ENSG00000163349_113905000_113925000
2,1,113905000,113910000,1,113925000,113930000,18.295846,1,113909957,113909958,...,0.058538,AnchorA,1,113929491,113929492,HIPK1-AS1,ENSG00000235527,-,CD8_Naive_1815-RH-1,1_113909957_ENSG00000235527_113905000_113925000
3,11,128295000,128300000,11,128520000,128525000,11.801121,11,128298735,128298736,...,0.130943,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128298735_ENSG00000276176_128295000_128520000
4,11,128300000,128305000,11,128520000,128525000,53.208704,11,128302574,128302575,...,0.122752,AnchorA,11,128522389,128522390,MIR6090,ENSG00000276176,+,CD8_Naive_1815-RH-1,11_128302574_ENSG00000276176_128300000_128520000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358,6,90275000,90280000,6,90295000,90300000,12.402266,6,90279405,90279406,...,0.083274,AnchorA,6,90296907,90296908,BACH2,ENSG00000112182,-,Monocyte_1831-RH-1,6_90279405_ENSG00000112182_90275000_90295000
6359,6,158650000,158655000,6,159040000,159045000,6.867517,6,159044944,159044945,...,0.122922,AnchorB,6,158650013,158650014,SYTL3,ENSG00000164674,+,Monocyte_1831-RH-1,6_159044944_ENSG00000164674_158650000_159040000
6360,6,159040000,159045000,6,159090000,159095000,8.984821,6,159044944,159044945,...,0.122922,AnchorA,6,159094092,159094093,AL356417.1,ENSG00000224478,+,Monocyte_1831-RH-1,6_159044944_ENSG00000224478_159040000_159090000
6361,6,159040000,159045000,6,159095000,159100000,26.156556,6,159044944,159044945,...,0.122922,AnchorA,6,159095822,159095823,AL035530.1,ENSG00000226032,-,Monocyte_1831-RH-1,6_159044944_ENSG00000226032_159040000_159095000


In [7]:
# all_df.to_csv(outdir + 'all_sgls.tsv', index=False, sep="\t")

#### Group lines with the same key

In [9]:
# https://stackoverflow.com/questions/36271413/pandas-merge-nearly-duplicate-rows-based-on-column-value
grouped_df = all_df.groupby('summary_id').agg({'chr_snp': 'first',
                                               '-log10_qval_loop':'min',
                                               'ppa_snp':'max',
                                               'genename': pd.Series.unique,
                                               'geneid': pd.Series.unique,
                                               'startA_loop': 'first',
                                               'endA_loop': 'first',
                                               'startB_loop': 'first',
                                               'endB_loop': 'first',
                                               'sample_name': pd.Series.unique,
                                              }).reset_index()

In [10]:
# 35 samples
grouped_df

,summary_id,chr_snp,-log10_qval_loop,ppa_snp,genename,geneid,startA_loop,endA_loop,startB_loop,endB_loop,sample_name
0,11_128298735_ENSG00000276176_128295000_128520000,11,11.801121,0.130943,MIR6090,ENSG00000276176,128295000,128300000,128520000,128525000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
1,11_128302574_ENSG00000254588_128300000_128525000,11,12.543043,0.122752,ETS1-AS1,ENSG00000254588,128300000,128305000,128525000,128530000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
2,11_128302574_ENSG00000276176_128300000_128520000,11,53.208704,0.122752,MIR6090,ENSG00000276176,128300000,128305000,128520000,128525000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
3,11_128302747_ENSG00000254588_128300000_128525000,11,12.543043,0.125901,ETS1-AS1,ENSG00000254588,128300000,128305000,128525000,128530000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
4,11_128302747_ENSG00000276176_128300000_128520000,11,53.208704,0.125901,MIR6090,ENSG00000276176,128300000,128305000,128520000,128525000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
5,11_128304885_ENSG00000254588_128300000_128525000,11,12.543043,0.127477,ETS1-AS1,ENSG00000254588,128300000,128305000,128525000,128530000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
6,11_128304885_ENSG00000276176_128300000_128520000,11,53.208704,0.127477,MIR6090,ENSG00000276176,128300000,128305000,128520000,128525000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
7,11_128308902_ENSG00000276176_128305000_128520000,11,7.669259,0.116469,MIR6090,ENSG00000276176,128305000,128310000,128520000,128525000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
8,12_56040361_ENSG00000135482_56040000_56115000,12,7.437393,0.350915,ZC3H10,ENSG00000135482,56040000,56045000,56115000,56120000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."
9,12_56040361_ENSG00000139641_56040000_56115000,12,7.437393,0.350915,ESYT1,ENSG00000139641,56040000,56045000,56115000,56120000,"[CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_N..."


In [11]:
# grouped_df.to_csv(outdir + "grouped_df.tsv", index=False, sep="\t")

#### Output df for Boostrap DataTable

In [12]:
output_df = grouped_df.copy(deep=True)
output_df = output_df.reindex(columns=['chr_snp', 'SNP BP', '-log10_qval_loop', 'ppa_snp', 
                                       'genename', 'geneid', 'startA_loop', 'endA_loop',
                                       'startB_loop', 'endB_loop', 'sample_name'])

output_df['sample_name'] = output_df['sample_name'].apply(lambda arr: ', '.join(str(name) for name in arr))
output_df['genename'] = output_df.apply(lambda row: row['genename'] + ' (' + row['geneid'] + ')', axis=1)
output_df = output_df.drop(columns=['geneid'])
output_df.sort_values(by=['chr_snp', 'startA_loop', 'startB_loop'], inplace=True)
output_df

,chr_snp,SNP BP,-log10_qval_loop,ppa_snp,genename,startA_loop,endA_loop,startB_loop,endB_loop,sample_name
39,1,NaN,6.195724,0.058538,CAPZA1 (ENSG00000116489),112615000,112620000,113905000,113910000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
40,1,NaN,18.295846,0.058538,HIPK1 (ENSG00000163349),113905000,113910000,113925000,113930000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
41,1,NaN,18.295846,0.058538,HIPK1-AS1 (ENSG00000235527),113905000,113910000,113925000,113930000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
48,2,NaN,6.067159,0.126805,DNAJC27 (ENSG00000115137),24915000,24920000,24970000,24975000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
49,2,NaN,6.067159,0.126805,DNAJC27-AS1 (ENSG00000224165),24915000,24920000,24970000,24975000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
50,2,NaN,6.067159,0.102268,DNAJC27 (ENSG00000115137),24915000,24920000,24970000,24975000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
51,2,NaN,6.067159,0.102268,DNAJC27-AS1 (ENSG00000224165),24915000,24920000,24970000,24975000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
52,2,NaN,5.989738,0.077211,DNAJC27 (ENSG00000115137),24920000,24925000,24970000,24975000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
53,2,NaN,5.989738,0.077211,DNAJC27-AS1 (ENSG00000224165),24920000,24925000,24970000,24975000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."
54,2,NaN,5.989738,0.114170,DNAJC27 (ENSG00000115137),24920000,24925000,24970000,24975000,"CD8_Naive_1815-RH-1, Naive_K_1829-RH-1, CD4_Na..."


In [13]:
# output_df.to_csv(outdir + "output_df.tsv", index=False, sep="\t")